Artem Chernitsa, B20-AI-01, a.chernitsa@innopolis.university

In [ ]:
!if [ $(pip freeze | grep -F gdown | wc -l) -eq 0 ]; then echo "not installed" && pip install gdown==4.3.1; else echo "already installed"; fi
!if [ $(ls | grep Assignment05 | wc -l) -eq 0 ]; then echo "no data" && gdown --folder --id 1b234EltvniwbIl-XOkMKnrW6kWM4jhhe; else echo "data is exists"; fi
!cp Assignment05/* .

already installed
no data
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder list
Processing file 11fR5B8t4fUfGO-JTq3duL2lwEPyy03N5 product_ids.csv
Processing file 1Pzs-zQihfo1DwfJdXkBShxHl-_-tpyMs products.csv
Processing file 10xws-aoTT58pzvAPADSn9fNWDDrkcUEn reviews.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=11fR5B8t4fUfGO-JTq3duL2lwEPyy03N5
To: /content/Assignment05/product_ids.csv
100% 799k/799k [00:00<00:00, 77.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Pzs-zQihfo1DwfJdXkBShxHl-_-tpyMs
To: /content/Assignment05/products.csv
100% 106M/106M [00:01<00:00, 78.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=10xws-aoTT58pzvAPADSn9fNWDDrkcUEn
To: /content/Assignment05/

### Install Spark on Colab

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget --continue https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz -O spark.tgz

--2023-05-01 10:03:13--  https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220400553 (210M) [application/x-gzip]
Saving to: ‘spark.tgz’

spark.tgz           100%[===================>] 210.19M  11.2MB/s    in 20s     

2023-05-01 10:03:34 (10.7 MB/s) - ‘spark.tgz’ saved [220400553/220400553]



In [ ]:
!tar xvzf spark.tgz

spark-3.0.3-bin-hadoop2.7/
spark-3.0.3-bin-hadoop2.7/NOTICE
spark-3.0.3-bin-hadoop2.7/kubernetes/
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/Dockerfile
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/Dockerfile
spark-3.0.3-bin-hadoop2.7/jars/
spark-3.0.3-bin-hadoop2.7/jars/jacks

In [ ]:
!pip install -q pyspark

In [ ]:
import os
import os.path

PATH = "/content"
SPARK_FOLDER = "spark-3.0.3-bin-hadoop2.7"
# SPARK_FOLDER = "spark-3.2.3-bin-hadoop2.7"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = os.path.join(PATH, SPARK_FOLDER)


# We also added paths for Spark binaries
os.environ["PATH"] += os.pathsep + os.path.join(PATH, SPARK_FOLDER, "bin")
os.environ["PATH"] += os.pathsep + os.path.join(PATH, SPARK_FOLDER, "sbin")



# os.environ['HADOOP_CONF_DIR'] = os.path.join(PATH, "hadoop-2.10.2/etc/hadoop")
# os.environ["HADOOP_HOME"] = os.path.join(PATH, "hadoop-2.10.2")
# os.environ['LD_LIBRARY_PATH']= os.path.join(PATH, "lib/native")

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

### Configure Spark

In [ ]:
! pip install -q pyngrok

In [ ]:
NGROK_TOKEN = "2OsgJSGgiviNpBDJ07JiY9rvkPZ_26p5GqzNTbL94Hspd9kdu"

ports ={
    "sparkiu" : 4050, 
}


config = '''
authtoken: {}
log: ngrok.log
version: '2'
tunnels:
  # to run jupyter notebook server
  sparkiu:
    addr: {}
    proto: http
'''.format(NGROK_TOKEN, ports["sparkiu"])

import yaml
ngrok_file = "ngrok.yml"
temp = yaml.safe_load(config)
with open(ngrok_file, 'w') as outfile:
    yaml.dump(temp, outfile, default_flow_style=False)

In [ ]:
!ngrok config --config "ngrok.yml" check

Valid configuration file at ngrok.yml


In [ ]:
start_ngrok = 'ngrok start --all --config {} --log=ngrok.log &'.format(ngrok_file)
get_ipython().system_raw(start_ngrok)

### Create Spark Session

In [ ]:
from pyspark.sql import SparkSession

# We will use this port number for monitoring Spark Jobs
port=4050

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Colab")\
        .config('spark.ui.port', str(port))\
        .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

spark

In [ ]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Colab>

### Spark App

In [ ]:
public_url = !curl -s http://localhost:4040/api/tunnels
public_url = !curl -s http://localhost:4040/api/tunnels
public_url

['{"tunnels":[{"name":"sparkiu","ID":"27f4ecb175979a70262b2f6ee94b60c3","uri":"/api/tunnels/sparkiu","public_url":"https://21fa-35-201-141-77.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}']

## Task on Spark MLlib

In [ ]:
!head -n 1 products.csv

title,date,feature,category,details,description,brand,rank,price,main_cat,product_id


### Read data

In [ ]:
import pandas as pd

with open("products.csv", "rb") as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())

# df = pd.read_csv("products.csv")
# df.info()
# df[["feature", "rank", "price", "product_id"]].head()
# df[(df["details"].notnull()) & (df["details"] != "{}")].head(30)

# df = df.iloc[11::50,:]
# df.info()


b'title,date,feature,category,details,description,brand,rank,price,main_cat,product_id\r\n'
b'BRAND NEW Learning Company Oregon Trail 5th Edition Jewel Case Swollen Dreams Fears Adventures Rivers,,"[\'If you have not opened the item, you can return the product to us for refund, credit or replacement within 30 days.\', \'Satisfaction Ensured.\', \'You l have to strategize and make quick decisions.\', \'And they l have to be right.\', \'Manufactured to the Highest Quality Available.\']","[\'Video Games\', \'PC\', \'Games\', \'</span></span></span>\']",,"[\'OREGON TRAIL 5TH EDITION (JEWEL CASE). Create Your Journey.  Explore the West! Live the Adventure! Survive the Trail! Westward, Ho! Are you ready to travel the Oregon Trail It going to take insight and careful planning to choose the right traveling companions, gear, and supplies. Even then, there are countless unpredictable hazards, like swollen rivers, poisonous creatures, and deadly diseases.Decide your occupation, who you l take wit

In [ ]:
import gc
import math

from pyspark.sql.types import StructType,\
                            ArrayType, \
                            StructField,\
                            IntegerType,\
                            DoubleType, \
                            StringType,\
                            BooleanType,\
                            TimestampType,\
                            DateType
import pyspark.sql.functions as F


from pyspark import keyword_only
from pyspark.sql import DataFrame

from pyspark.ml import Transformer, Pipeline
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable

from pyspark.ml.feature import \
    StringIndexer, \
    Word2Vec, \
    MinMaxScaler, \
    VectorAssembler, \
    VectorIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import \
    RandomForestClassifier, \
    MultilayerPerceptronClassifier
from pyspark.ml.recommendation import ALS
from pyspark.mllib.evaluation import MulticlassMetrics

SID = 12

product_ids_schema = StructType([
    StructField("product_id", StringType()),
])

################################################################################
# TODO READ AS TEXT, REPLACE "" TO "
# THEN CONVERT TO DATAFRAME WITH OPTIONS AS BELOW

product_ids = products = reviews = None

product_ids = spark.read \
    .option("header", True) \
    .option("mode", "DROPMALFORMED") \
    .option("wholeFile", True) \
    .option("multiLine", True) \
    .schema(product_ids_schema) \
    .csv("product_ids.csv")
rdd = product_ids.rdd
product_ids = rdd.zipWithIndex() \
    .map(lambda x: (x[0], x[1] - (SID - 1))) \
    .filter(lambda x: x[1] % 50 == 0) \
    .map(lambda x: x[0]) \
    .toDF()
product_ids.limit(10).toPandas().head()

,product_id
0,0439381673
1,1613170483
2,9054675152
3,9861767304
4,B000009QD1


In [ ]:
reviews_schema = StructType([
    StructField("sentiment", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("reviewTime", StringType()),
    StructField("reviewerID", StringType()),
    StructField("reviewerName", StringType()),
    StructField("reviewText", StringType()),
    StructField("unixReviewTime", StringType()),
    StructField("vote", IntegerType()),
    StructField("image", StringType()),
    StructField("product_id", StringType()),
])

reviews = spark.read \
    .option("header", True) \
    .option("mode", "DROPMALFORMED") \
    .option("wholeFile", True) \
    .option("multiLine", True) \
    .schema(reviews_schema) \
    .csv("reviews.csv")
reviews = reviews.join(product_ids, "product_id", "inner")
reviews.limit(10).toPandas().head()

,product_id,sentiment,rating,reviewTime,reviewerID,reviewerName,reviewText,unixReviewTime,vote,image
0,B00002STH3,1,5,"08 2, 2016",AJKKLETPK6IJX,Claude curry,Thank you for this.. This was a hit in my hous...,1470096000,0,None
1,B00002STH3,1,4,"09 23, 2015",A2NMICZ97JQ8QO,Justin,Surprisingly fun and underrated. Pretty easy w...,1442966400,0,None
2,B00002STH3,1,5,"12 26, 2014",A1XA6PJNF37LPM,Salem Alhubail,Awesome,1419552000,0,None
3,B00002STH3,1,4,"05 2, 2014",ARX0Y3X8WZL6V,Charles Patton,"If you are solely into racing battle games, th...",1398988800,0,None
4,B00002STH3,1,4,"02 12, 2014",A39QYCBKS4D6E7,Tim Case,Good game to play multiplayer. I remember rent...,1392163200,0,None


In [ ]:
# fin = open("products.csv", "rt")
# fout = open("products_out.csv", "wt")

# for line in fin:
# 	fout.write(line.replace('""', '"'))

# fin.close()
# fout.close()

In [ ]:
products_schema = StructType([
    StructField("title", StringType()),
    StructField("date", StringType()),
    StructField("feature", StringType()),
    StructField("category", StringType()),
    StructField("details", StringType()),
    StructField("description", StringType()),
    StructField("brand", StringType()),
    StructField("rank", StringType()),
    StructField("price", StringType()),
    StructField("main_cat", StringType()),
    StructField("product_id", StringType()),
])

products = spark.read \
    .option("header", True) \
    .option("mode", "DROPMALFORMED") \
    .option("wholeFile", True) \
    .option("multiLine", True) \
    .schema(products_schema) \
    .csv("products.csv")

products = products.join(product_ids, "product_id", "inner")
products.limit(10).toPandas().head()

,product_id,title,date,feature,category,details,description,brand,rank,price,main_cat
0,B00002STH3,Extreme G,None,[],"['Video Games', 'Retro Gaming & Microconsoles'...",{},"""['stickers over back label', """"A multi-player...",Acclaim Studios,"['>#28,688 in Video Games (See Top 100 in Vide...",$11.46,Video Games
1,B00004SVZU,Primal Rage - Sega Saturn,None,['Brutal Combat Action!'],"['Video Games', 'Retro Gaming & Microconsoles'...",{},['Disc(s) only. Ships in generic case. Disc(s)...,by\n \n Midway,"['>#73,278 in Video Games (See Top 100 in Vide...",None,Video Games
2,B0012GMU86,Tony Hawk's Project 8 Enhanced (Platinum Hits),None,['Brings Skateboarding to Life - the most adva...,"['Video Games', 'Xbox 360', 'Games', '</span><...",{},"""[""""Tony Hawk's Project 8 Enhanced Platinum Hi...",by\n \n Activision,"['>#56,424 in Video Games (See Top 100 in Vide...",None,Video Games
3,B00WL10U1K,ModFreakz Shell/Button Kit Hydro Dipped Collec...,None,"""['Shell for PS4 Gen 1,2 V1 controller. Refer ...","['Video Games', 'PlayStation 4', 'Accessories'...",{},['Each one of our custom designed and created ...,Mod Freakz,"['>#70,681 in Video Games (See Top 100 in Vide...",None,Video Games
4,B00005K3WW,Sony PlayStation Tennis Game,None,"['ALL TIME CLASSIC', 'GREAT TENNIS GAME', 'FOR...","['Video Games', 'Retro Gaming & Microconsoles'...",{},['Smashing fun for everyone! Choose from 16 d...,by\n \n Agetec,"['>#100,915 in Video Games (See Top 100 in Vid...",None,Video Games


In [ ]:
def read_files():
    global product_ids
    global products
    global reviews

    product_ids = spark.read \
        .option("header", True) \
        .option("mode", "DROPMALFORMED") \
        .option("wholeFile", True) \
        .option("multiLine", True) \
        .schema(product_ids_schema) \
        .csv("product_ids.csv")
    rdd = product_ids.rdd
    product_ids = rdd.zipWithIndex() \
        .map(lambda x: (x[0], x[1] - (SID - 1))) \
        .filter(lambda x: x[1] % 50 == 0) \
        .map(lambda x: x[0]) \
        .toDF()

    products = spark.read \
        .option("header", True) \
        .option("mode", "DROPMALFORMED") \
        .option("wholeFile", True) \
        .option("multiLine", True) \
        .schema(products_schema) \
        .csv("products.csv")
    products = products.join(product_ids, "product_id", "inner")


    reviews = spark.read \
        .option("header", True) \
        .option("mode", "DROPMALFORMED") \
        .option("wholeFile", True) \
        .option("multiLine", True) \
        .schema(reviews_schema) \
        .csv("reviews.csv")
    reviews = reviews.join(product_ids, "product_id", "inner")

In [ ]:
def clear_cache():
    gc.collect()

    spark.catalog.clearCache()
    for (id, rdd) in sc._jsc.getPersistentRDDs().items():
        rdd.unpersist()
        print("Unpersisted {} rdd".format(id))

In [ ]:
def clear_all():
    global products
    global product_ids
    global reviews

    del products
    del product_ids
    del reviews

    clear_cache()

### Task 1. Regression

#### Preprocess

In [ ]:
clear_all()
read_files()

In [ ]:
# Clear
# delete $ from price, make double

# Remove array-like types
# feature, category, description, rank

# Remove dictionary-like types
# details

# Decompose date field
# keep only month and year

# Encoding non-numerical features using TF-IDF
# title, brand, main_cat

In [ ]:
products = products.na.drop(subset=["title", "brand", "date", "main_cat"])
products.limit(5).show()

,product_id,title,date,feature,category,details,description,brand,rank,price,main_cat
0,B012KPIRY8,Vivi Audio&reg; 3.8mm + 4.5mm Security Screwdr...,"July 26, 2015",['3.8mm opens game cartridges for Original Nin...,"['Video Games', 'Accessories']",{},['<br> 3.8mm + 4.5mm Security Screwdriver Tool...,Vivi Audio,"['>#149,435 in Electronics (See Top 100 in Ele...",$4.99,Computers
1,B00XH47J7I,Old Skool Classic Sega Genesis Controller- 6-B...,"May 11, 2015","['Eight-way directional pad', 'Classic six-but...","['Video Games', 'Retro Gaming & Microconsoles'...",{},['Replace your broken or lost controller with ...,Old Skool,"['>#31,876 in Electronics (See Top 100 in Elec...",$6.99,Computers
2,B01ASK7YSM,SLLEA US Home Travel AC Charger for Nintendo D...,"January 19, 2016",['100% new high quality (NON-OEM) US Travel Ch...,"['Video Games', 'Accessories']",{},['Decription: \n<br> INPUT: AC 100-250V 50/60H...,SLLEA,"['>#23,056 in Computers & Accessories (See top...",$8.99,Computers
3,B00B1NZN5M,AULA 120560 Ghost Shark Expert Gaming Mouse,"April 16, 2013",['Laser wired USB gaming mouse with 800-2000DP...,"['Video Games', 'PC', 'Accessories', 'Gaming M...",{},"['High-end Laser engine, precise positioning T...",AULA,['>#790 in Video Games > PC Games > PC Game Ha...,$4.12,Computers
4,B00NICUF16,Generic Leopard Full Shell + Buttons Kits Game...,"September 12, 2014",[],"['Video Games', 'Accessories']",{},[],Kukemo,"['>#1,180,511 in Electronics (See Top 100 in E...",None,Computers


In [ ]:
products.summary()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_id   96 non-null     object
 1   title        96 non-null     object
 2   date         96 non-null     object
 3   feature      96 non-null     object
 4   category     96 non-null     object
 5   details      96 non-null     object
 6   description  96 non-null     object
 7   brand        96 non-null     object
 8   rank         96 non-null     object
 9   price        35 non-null     object
 10  main_cat     96 non-null     object
dtypes: object(11)
memory usage: 8.4+ KB


In [ ]:
products = products \
    .drop("feature", "category", "description", "rank") \
    .drop("details")
products.limit(5).show()

,product_id,title,date,brand,price,main_cat
0,B012KPIRY8,Vivi Audio&reg; 3.8mm + 4.5mm Security Screwdr...,"July 26, 2015",Vivi Audio,$4.99,Computers
1,B00XH47J7I,Old Skool Classic Sega Genesis Controller- 6-B...,"May 11, 2015",Old Skool,$6.99,Computers
2,B01ASK7YSM,SLLEA US Home Travel AC Charger for Nintendo D...,"January 19, 2016",SLLEA,$8.99,Computers
3,B00B1NZN5M,AULA 120560 Ghost Shark Expert Gaming Mouse,"April 16, 2013",AULA,$4.12,Computers
4,B00NICUF16,Generic Leopard Full Shell + Buttons Kits Game...,"September 12, 2014",Kukemo,None,Computers


In [ ]:
products = products \
    .withColumn("date_month", F.regexp_extract(F.col("date"), "([A-z]+)", 1)) \
    .withColumn("date_year", F.regexp_extract(F.col("date"), ",\s([0-9]+)", 1)) \
    .drop("date")
products = products.withColumn("date_year", F.col("date_year").cast("int"))
products.limit(5).show()

,product_id,title,brand,price,main_cat,date_month,date_year
0,B012KPIRY8,Vivi Audio&reg; 3.8mm + 4.5mm Security Screwdr...,Vivi Audio,$4.99,Computers,July,2015
1,B00XH47J7I,Old Skool Classic Sega Genesis Controller- 6-B...,Old Skool,$6.99,Computers,May,2015
2,B01ASK7YSM,SLLEA US Home Travel AC Charger for Nintendo D...,SLLEA,$8.99,Computers,January,2016
3,B00B1NZN5M,AULA 120560 Ghost Shark Expert Gaming Mouse,AULA,$4.12,Computers,April,2013
4,B00NICUF16,Generic Leopard Full Shell + Buttons Kits Game...,Kukemo,None,Computers,September,2014


In [ ]:
MONTHS = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

# products = products.rdd.map(
#     lambda x: (x.product_id, x.title, x.brand, x.price, x.main_cat, MONTHS[x.date_month], x.date_year)
# ).toDF(products.columns)
encode_month_udf = F.udf(lambda x: MONTHS[x], IntegerType())
products = products \
    .withColumn("date_month", encode_month_udf(F.col("date_month")))
products.limit(5).show()

,product_id,title,brand,price,main_cat,date_month,date_year
0,B012KPIRY8,Vivi Audio&reg; 3.8mm + 4.5mm Security Screwdr...,Vivi Audio,$4.99,Computers,7,2015
1,B00XH47J7I,Old Skool Classic Sega Genesis Controller- 6-B...,Old Skool,$6.99,Computers,5,2015
2,B01ASK7YSM,SLLEA US Home Travel AC Charger for Nintendo D...,SLLEA,$8.99,Computers,1,2016
3,B00B1NZN5M,AULA 120560 Ghost Shark Expert Gaming Mouse,AULA,$4.12,Computers,4,2013
4,B00NICUF16,Generic Leopard Full Shell + Buttons Kits Game...,Kukemo,None,Computers,9,2014


In [ ]:
class SinTransformer(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    input_col = Param(Params._dummy(), "input_col", "input column name.", typeConverter=TypeConverters.toString)
    output_col = Param(Params._dummy(), "output_col", "output column name.", typeConverter=TypeConverters.toString)

    @keyword_only
    def __init__(self, input_col: str = "input", output_col: str = "output"):
        super().__init__()
        self._setDefault(input_col=None, output_col=None)
        kwargs = self._input_kwargs
        self.set_params(**kwargs)

    @keyword_only
    def set_params(self, input_col: str = "input", output_col: str = "output"):
        kwargs = self._input_kwargs
        self._set(**kwargs)

    def get_input_col(self):
        return self.getOrDefault(self.input_col)

    def get_output_col(self):
        return self.getOrDefault(self.output_col)

    def _transform(self, df: DataFrame):
        input_col = self.get_input_col()
        output_col = self.get_output_col()
        transform_udf = F.udf(lambda x: math.sin(2 * math.pi * x / 12), DoubleType())
        return df.withColumn(output_col, transform_udf(input_col))

class CosTransformer(SinTransformer):
    def _transform(self, df: DataFrame):
        input_col = self.get_input_col()
        output_col = self.get_output_col()
        transform_udf = F.udf(lambda x: math.cos(2 * math.pi * x / 12), DoubleType())
        return df.withColumn(output_col, transform_udf(input_col))


sin_transformer = SinTransformer(input_col="date_month", output_col="sin_month")
cos_transformer = CosTransformer(input_col="date_month", output_col="cos_month")

pipeline = Pipeline(stages=[sin_transformer, cos_transformer])
model = pipeline.fit(products)
products = model.transform(products)

products.limit(5).show()

,product_id,title,brand,price,main_cat,date_month,date_year,sin_month,cos_month
0,B012KPIRY8,Vivi Audio&reg; 3.8mm + 4.5mm Security Screwdr...,Vivi Audio,$4.99,Computers,7,2015,-0.500000,-8.660254e-01
1,B00XH47J7I,Old Skool Classic Sega Genesis Controller- 6-B...,Old Skool,$6.99,Computers,5,2015,0.500000,-8.660254e-01
2,B01ASK7YSM,SLLEA US Home Travel AC Charger for Nintendo D...,SLLEA,$8.99,Computers,1,2016,0.500000,8.660254e-01
3,B00B1NZN5M,AULA 120560 Ghost Shark Expert Gaming Mouse,AULA,$4.12,Computers,4,2013,0.866025,-5.000000e-01
4,B00NICUF16,Generic Leopard Full Shell + Buttons Kits Game...,Kukemo,None,Computers,9,2014,-1.000000,-1.836970e-16


In [ ]:
products = products.drop("date_month")
products.limit(5).show()

,product_id,title,brand,price,main_cat,date_year,sin_month,cos_month
0,B012KPIRY8,Vivi Audio&reg; 3.8mm + 4.5mm Security Screwdr...,Vivi Audio,$4.99,Computers,2015,-0.500000,-8.660254e-01
1,B00XH47J7I,Old Skool Classic Sega Genesis Controller- 6-B...,Old Skool,$6.99,Computers,2015,0.500000,-8.660254e-01
2,B01ASK7YSM,SLLEA US Home Travel AC Charger for Nintendo D...,SLLEA,$8.99,Computers,2016,0.500000,8.660254e-01
3,B00B1NZN5M,AULA 120560 Ghost Shark Expert Gaming Mouse,AULA,$4.12,Computers,2013,0.866025,-5.000000e-01
4,B00NICUF16,Generic Leopard Full Shell + Buttons Kits Game...,Kukemo,None,Computers,2014,-1.000000,-1.836970e-16


In [ ]:
products = products.withColumn("title", F.split("title", " "))

word2Vec = Word2Vec(vectorSize=20, seed=42, inputCol="title", outputCol="title_encoded")
model = word2Vec.fit(products)
products = model.transform(products)
products = products.drop("title")
products.limit(5).show()

,product_id,brand,price,main_cat,date_year,sin_month,cos_month,title_encoded
0,B012KPIRY8,Vivi Audio,$4.99,Computers,2015,-0.500000,-8.660254e-01,"[0.00012524393322463665, 0.0017590726622276835..."
1,B00XH47J7I,Old Skool,$6.99,Computers,2015,0.500000,-8.660254e-01,"[0.002119755579365624, 0.0007226524046725697, ..."
2,B01ASK7YSM,SLLEA,$8.99,Computers,2016,0.500000,8.660254e-01,"[-0.00015408358012791723, 0.003317624951402346..."
3,B00B1NZN5M,AULA,$4.12,Computers,2013,0.866025,-5.000000e-01,"[-0.0024622378072568347, 0.0013917088508605957..."
4,B00NICUF16,Kukemo,None,Computers,2014,-1.000000,-1.836970e-16,"[0.001024155890003645, -0.004201856175703661, ..."


In [ ]:
string_indexer = StringIndexer(inputCol="brand", outputCol="brand_encoded")
string_indexer = string_indexer.fit(products)
products = string_indexer.transform(products)
products = products.drop("brand")
products.limit(5).show()

,product_id,price,main_cat,date_year,sin_month,cos_month,title_encoded,brand_encoded
0,B012KPIRY8,$4.99,Computers,2015,-0.500000,-8.660254e-01,"[0.00012524393322463665, 0.0017590726622276835...",7.0
1,B00XH47J7I,$6.99,Computers,2015,0.500000,-8.660254e-01,"[0.002119755579365624, 0.0007226524046725697, ...",54.0
2,B01ASK7YSM,$8.99,Computers,2016,0.500000,8.660254e-01,"[-0.00015408358012791723, 0.003317624951402346...",60.0
3,B00B1NZN5M,$4.12,Computers,2013,0.866025,-5.000000e-01,"[-0.0024622378072568347, 0.0013917088508605957...",10.0
4,B00NICUF16,None,Computers,2014,-1.000000,-1.836970e-16,"[0.001024155890003645, -0.004201856175703661, ...",40.0


In [ ]:
string_indexer = StringIndexer(inputCol="main_cat", outputCol="main_cat_encoded")
string_indexer = string_indexer.fit(products)
products = string_indexer.transform(products)
products = products.drop("main_cat")
products.limit(5).show()

,product_id,price,date_year,sin_month,cos_month,title_encoded,brand_encoded,main_cat_encoded
0,B012KPIRY8,$4.99,2015,-0.500000,-8.660254e-01,"[0.00012524393322463665, 0.0017590726622276835...",7.0,0.0
1,B00XH47J7I,$6.99,2015,0.500000,-8.660254e-01,"[0.002119755579365624, 0.0007226524046725697, ...",54.0,0.0
2,B01ASK7YSM,$8.99,2016,0.500000,8.660254e-01,"[-0.00015408358012791723, 0.003317624951402346...",60.0,0.0
3,B00B1NZN5M,$4.12,2013,0.866025,-5.000000e-01,"[-0.0024622378072568347, 0.0013917088508605957...",10.0,0.0
4,B00NICUF16,None,2014,-1.000000,-1.836970e-16,"[0.001024155890003645, -0.004201856175703661, ...",40.0,0.0


In [ ]:
assembler = VectorAssembler(inputCols=["date_year"], outputCol="date_year_array")
products = assembler.transform(products)

mmScaler = MinMaxScaler(inputCol="date_year_array", outputCol="date_year_encoded")
model = mmScaler.fit(products)
products = model.transform(products)
products = products.drop("date_year_array").drop("date_year")
products.limit(5).show()

,product_id,price,sin_month,cos_month,title_encoded,brand_encoded,main_cat_encoded,date_year_encoded
0,B012KPIRY8,$4.99,-0.500000,-8.660254e-01,"[0.00012524393322463665, 0.0017590726622276835...",7.0,0.0,[0.9333333333333333]
1,B00XH47J7I,$6.99,0.500000,-8.660254e-01,"[0.002119755579365624, 0.0007226524046725697, ...",54.0,0.0,[0.9333333333333333]
2,B01ASK7YSM,$8.99,0.500000,8.660254e-01,"[-0.00015408358012791723, 0.003317624951402346...",60.0,0.0,[0.9555555555555556]
3,B00B1NZN5M,$4.12,0.866025,-5.000000e-01,"[-0.0024622378072568347, 0.0013917088508605957...",10.0,0.0,[0.888888888888889]
4,B00NICUF16,None,-1.000000,-1.836970e-16,"[0.001024155890003645, -0.004201856175703661, ...",40.0,0.0,[0.9111111111111111]


In [ ]:
products.summary()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         96 non-null     object 
 1   price              35 non-null     object 
 2   sin_month          96 non-null     float64
 3   cos_month          96 non-null     float64
 4   title_encoded      96 non-null     object 
 5   brand_encoded      96 non-null     float64
 6   main_cat_encoded   96 non-null     float64
 7   date_year_encoded  96 non-null     object 
dtypes: float64(4), object(4)
memory usage: 6.1+ KB


In [ ]:
products_nan = products.filter(F.col("price").isNull())
products_nan.limit(5).show()

,product_id,price,sin_month,cos_month,title_encoded,brand_encoded,main_cat_encoded,date_year_encoded
0,B00NICUF16,None,-1.0,-1.836970e-16,"[0.001024155890003645, -0.004201856175703661, ...",40.0,0.0,[0.9111111111111111]
1,B019C35JEW,None,0.5,-8.660254e-01,"[-0.0014461122031900143, 0.000911039388960316,...",45.0,0.0,[0.9555555555555556]
2,B00C6VM9PA,None,-0.5,-8.660254e-01,"[0.004880883730947971, 0.0005251489579677582, ...",53.0,0.0,[0.888888888888889]
3,B00LGABAWA,None,-0.5,8.660254e-01,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.0,1.0,[0.9333333333333333]
4,B01GBZ3DFG,None,0.5,-8.660254e-01,"[-0.005390085105318576, -0.0028545151309420664...",30.0,0.0,[0.9555555555555556]


In [ ]:
products_nan.summary()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         61 non-null     object 
 1   price              0 non-null      object 
 2   sin_month          61 non-null     float64
 3   cos_month          61 non-null     float64
 4   title_encoded      61 non-null     object 
 5   brand_encoded      61 non-null     float64
 6   main_cat_encoded   61 non-null     float64
 7   date_year_encoded  61 non-null     object 
dtypes: float64(4), object(4)
memory usage: 3.9+ KB


In [ ]:
products = products \
    .filter(F.col("price").rlike("\$\d+[\.]?\d+")) \
    .withColumn("price", F.regexp_extract(F.col("price"), "\$(\d+[\.]?\d+)", 1)) \
    .withColumn("price", F.col("price").cast(DoubleType()))
products.limit(5).show()

,product_id,price,sin_month,cos_month,title_encoded,brand_encoded,main_cat_encoded,date_year_encoded
0,B012KPIRY8,4.99,-0.500000,-0.866025,"[0.00012524393322463665, 0.0017590726622276835...",7.0,0.0,[0.9333333333333333]
1,B00XH47J7I,6.99,0.500000,-0.866025,"[0.002119755579365624, 0.0007226524046725697, ...",54.0,0.0,[0.9333333333333333]
2,B01ASK7YSM,8.99,0.500000,0.866025,"[-0.00015408358012791723, 0.003317624951402346...",60.0,0.0,[0.9555555555555556]
3,B00B1NZN5M,4.12,0.866025,-0.500000,"[-0.0024622378072568347, 0.0013917088508605957...",10.0,0.0,[0.888888888888889]
4,B00WNAOLE6,14.99,0.866025,-0.500000,"[0.002589470852399245, -0.00078283845713096, 0...",0.0,0.0,[0.9333333333333333]


In [ ]:
products.summary()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         33 non-null     object 
 1   price              33 non-null     float64
 2   sin_month          33 non-null     float64
 3   cos_month          33 non-null     float64
 4   title_encoded      33 non-null     object 
 5   brand_encoded      33 non-null     float64
 6   main_cat_encoded   33 non-null     float64
 7   date_year_encoded  33 non-null     object 
dtypes: float64(5), object(3)
memory usage: 2.2+ KB


#### Build model

In [ ]:
# For training the model, use only the products whose prices are known (not missing).

In [ ]:
assembler = VectorAssembler(
    inputCols=["sin_month", "cos_month", "title_encoded", "brand_encoded", "main_cat_encoded", "date_year_encoded"],
    outputCol="features"
)
pipeline = Pipeline(stages=[assembler])
model_pipe = pipeline.fit(products)
data = model_pipe.transform(products)
data.limit(5).show()

,product_id,price,sin_month,cos_month,title_encoded,brand_encoded,main_cat_encoded,date_year_encoded,features
0,B012KPIRY8,4.99,-0.500000,-0.866025,"[0.00012524393322463665, 0.0017590726622276835...",7.0,0.0,[0.9333333333333333],"[-0.4999999999999997, -0.8660254037844388, 0.0..."
1,B00XH47J7I,6.99,0.500000,-0.866025,"[0.002119755579365624, 0.0007226524046725697, ...",54.0,0.0,[0.9333333333333333],"[0.49999999999999994, -0.8660254037844387, 0.0..."
2,B01ASK7YSM,8.99,0.500000,0.866025,"[-0.00015408358012791723, 0.003317624951402346...",60.0,0.0,[0.9555555555555556],"[0.49999999999999994, 0.8660254037844387, -0.0..."
3,B00B1NZN5M,4.12,0.866025,-0.500000,"[-0.0024622378072568347, 0.0013917088508605957...",10.0,0.0,[0.888888888888889],"[0.8660254037844387, -0.4999999999999998, -0.0..."
4,B00WNAOLE6,14.99,0.866025,-0.500000,"[0.002589470852399245, -0.00078283845713096, 0...",0.0,0.0,[0.9333333333333333],"[0.8660254037844387, -0.4999999999999998, 0.00..."


In [ ]:
data = data.select("price", "features")
data = data.withColumnRenamed("price", "label")

# (trainingData, testData) = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
data.summary()

summary,label
count,33
mean,9.475454545454545
stddev,5.9702382128506235
min,0.99
25%,4.99
50%,7.98
75%,14.56
max,29.99


If we divide the sample into training and test, then we will use only part of the data to train the model, which means that our model will be less accurate than if we used all the available data. This is especially important in tasks where there is little data available, or when the data is not representative.

Also, splitting the sample can lead to an imbalance of data in the training and test samples, which can lead to a distortion of the evaluation of the model's performance. For example, if the sample contains a small number of objects with a high price, then when randomly partitioned, these objects may fall only into the training or only into the test sample, which will affect the evaluation of the model's performance.

In general, the absence of splitting the sample into training and test allows you to use all available data to train the model and get a more accurate performance estimate for the entire sample.

In [ ]:
# Create Linear Regression Model
lr = LinearRegression()

# Add the last indexer with the model to the pipeline
pipeline = Pipeline(stages=[lr])

# Fit the data to the pipeline stages
model = pipeline.fit(data)

# Transform the data (Prediction)
predictions = model.transform(data)

# Display the predictions
predictions.show()

# Evaluate the performance of the model
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
| 4.99|[-0.4999999999999...|6.7327592211518805|
| 6.99|[0.49999999999999...| 9.415446124143909|
| 8.99|[0.49999999999999...| 10.21016404384482|
| 4.12|[0.86602540378443...|4.1283585474519136|
|14.99|[0.86602540378443...| 16.70881777286908|
| 0.99|[0.49999999999999...|1.2449755702336311|
| 3.99|[-0.4999999999999...| 5.810397248474594|
| 9.99|[-0.5000000000000...| 7.786274055279652|
|16.99|[0.86602540378443...|15.163450930398152|
|15.99|[0.86602540378443...|13.789523663649494|
|29.99|[-2.4492935982947...|24.513086320145387|
| 7.99|[-0.8660254037844...| 6.242531695124963|
| 1.54|[-1.0,-1.83697019...| 2.553728923846954|
|  5.0|(25,[0,1,22,23,24...| 4.086563015999751|
| 2.54|[-0.8660254037844...| 5.044844938298045|
|13.99|[1.0,6.1232339957...| 12.85985395534841|
| 9.99|[-0.8660254037844...| 9.395124971953908|
| 9.99|[1.0,6.1232339957...|  11.8137027

In [ ]:
# Create Linear Regression Model
lr_2 = LinearRegression(maxIter=42)

# Add the last indexer with the model to the pipeline
pipeline_2 = Pipeline(stages=[lr_2])

# Fit the data to the pipeline stages
model_2 = pipeline.fit(data)

# Transform the data (Prediction)
predictions_2 = model_2.transform(data)

# Display the predictions
predictions_2.show()

# Evaluate the performance of the model
evaluator_2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse_2 = evaluator_2.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_2)

+-----+--------------------+------------------+
|label|            features|        prediction|
+-----+--------------------+------------------+
| 4.99|[-0.4999999999999...| 6.732759221151973|
| 6.99|[0.49999999999999...| 9.415446124143905|
| 8.99|[0.49999999999999...|10.210164043844848|
| 4.12|[0.86602540378443...|  4.12835854745212|
|14.99|[0.86602540378443...|16.708817772869047|
| 0.99|[0.49999999999999...|1.2449755702337626|
| 3.99|[-0.4999999999999...| 5.810397248474587|
| 9.99|[-0.5000000000000...|  7.78627405527957|
|16.99|[0.86602540378443...|15.163450930398284|
|15.99|[0.86602540378443...|13.789523663649419|
|29.99|[-2.4492935982947...| 24.51308632014537|
| 7.99|[-0.8660254037844...| 6.242531695124953|
| 1.54|[-1.0,-1.83697019...| 2.553728923847139|
|  5.0|(25,[0,1,22,23,24...| 4.086563015999769|
| 2.54|[-0.8660254037844...| 5.044844938298098|
|13.99|[1.0,6.1232339957...|12.859853955348484|
| 9.99|[-0.8660254037844...| 9.395124971953969|
| 9.99|[1.0,6.1232339957...|11.813702703

#### Prediction

In [ ]:
assembler = VectorAssembler(
    inputCols=["sin_month", "cos_month", "title_encoded", "brand_encoded", "main_cat_encoded", "date_year_encoded"],
    outputCol="features"
)
pipeline = Pipeline(stages=[assembler])
model_pipe = pipeline.fit(products_nan)
data_nan = model_pipe.transform(products_nan)

# data_nan.show()

data_nan = data_nan.select("price", "features")
data_nan = data_nan.withColumnRenamed("price", "label")

# data_nan.show()

predictions = model.transform(data_nan)
predictions.show()

+-----+--------------------+-------------------+
|label|            features|         prediction|
+-----+--------------------+-------------------+
| null|[-1.0,-1.83697019...| 20.045108044692082|
| null|[0.49999999999999...| 11.662614040613192|
| null|[-0.4999999999999...|-16.365075242630407|
| null|(25,[0,1,22,23,24...|  6.050187836868261|
| null|[0.49999999999999...| 11.623654385827635|
| null|[-0.4999999999999...|    10.966423424709|
| null|[-0.4999999999999...| 11.309619897608972|
| null|[0.49999999999999...| 11.294087891553112|
| null|[-1.0,-1.83697019...| 17.650058985623115|
| null|[0.49999999999999...|  9.298051323330402|
| null|[0.49999999999999...| 13.666832396287703|
| null|[-2.4492935982947...|  35.10226824946571|
| null|[1.0,6.1232339957...| 16.097018054635665|
| null|[0.86602540378443...| 18.879144719639722|
| null|[0.86602540378443...|  17.83831482865413|
| null|[-0.5000000000000...| 13.851044208376358|
| null|[-0.5000000000000...|  6.689103971675632|
| null|[-0.866025403

### Task 2. Sentiment Analysis

In [ ]:
read_files()
reviews.limit(5).show()

,product_id,sentiment,rating,reviewTime,reviewerID,reviewerName,reviewText,unixReviewTime,vote,image
0,B00002STH3,1,5,"08 2, 2016",AJKKLETPK6IJX,Claude curry,Thank you for this.. This was a hit in my hous...,1470096000,0,None
1,B00002STH3,1,4,"09 23, 2015",A2NMICZ97JQ8QO,Justin,Surprisingly fun and underrated. Pretty easy w...,1442966400,0,None
2,B00002STH3,1,5,"12 26, 2014",A1XA6PJNF37LPM,Salem Alhubail,Awesome,1419552000,0,None
3,B00002STH3,1,4,"05 2, 2014",ARX0Y3X8WZL6V,Charles Patton,"If you are solely into racing battle games, th...",1398988800,0,None
4,B00002STH3,1,4,"02 12, 2014",A39QYCBKS4D6E7,Tim Case,Good game to play multiplayer. I remember rent...,1392163200,0,None


In [ ]:
reviews = reviews.drop("product_id", "reviewTime", "reviewerID", "reviewerName", "unixReviewTime", "image")
reviews.limit(5).show()

,sentiment,rating,reviewText,vote
0,1,5,Great way for kids to get excited about learni...,0
1,1,1,I was sooooo excited to find this game!! And i...,4
2,0,1,"What a bore! I love this game, but so far only...",4
3,-1,1,This item would not work on my Windows 8 compu...,0
4,-1,1,My son got this for Christmas and we couldn't ...,0


In [ ]:
reviews = reviews.withColumn("reviewText", F.split("reviewText", " "))

word2Vec = Word2Vec(vectorSize=20, seed=42, inputCol="reviewText", outputCol="reviewText_encoded")
model = word2Vec.fit(reviews)
reviews = model.transform(reviews)
reviews = reviews.drop("reviewText")
reviews.limit(5).show()

,sentiment,rating,vote,reviewText_encoded
0,1,5,0,"[-0.013172204988465334, -0.01966195465065539, ..."
1,1,1,4,"[0.008816595473382177, -0.02335511125247216, 0..."
2,0,1,4,"[0.014142050756190933, -0.0067899450730306635,..."
3,-1,1,0,"[-0.01381436545545092, 0.019943410638146676, 0..."
4,-1,1,0,"[-0.016544603631787357, -0.015580453632754229,..."


In [ ]:
reviews.summary()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sentiment           866 non-null    int32 
 1   rating              866 non-null    int32 
 2   vote                866 non-null    int32 
 3   reviewText_encoded  866 non-null    object
dtypes: int32(3), object(1)
memory usage: 17.0+ KB


In [ ]:
# Encode label for classification (classes must be positive integers)
reviews = reviews.withColumn("sentiment", F.col("sentiment") + 1)
reviews.show()

+---------+------+----+--------------------+
|sentiment|rating|vote|  reviewText_encoded|
+---------+------+----+--------------------+
|        2|     5|   0|[-0.0131722049884...|
|        2|     1|   4|[0.00881659547338...|
|        1|     1|   4|[0.01414205075619...|
|        0|     1|   0|[-0.0138143654554...|
|        0|     1|   0|[-0.0165446036317...|
|        2|     5|   0|[3.93529804811502...|
|        1|     3|   0|[0.00543858343735...|
|        0|     1|   0|[0.00656521735856...|
|        0|     3|   0|[-0.0084159789031...|
|        2|     5|   0|[-0.0197428670799...|
|        2|     5|   0|[-4.7050964044915...|
|        0|     1|   0|[0.02123948952922...|
|        2|     4|   2|[-0.0173882473415...|
|        0|     1|   0|[-0.0149817757875...|
|        0|     1|   0|[-0.0106316566719...|
|        2|     5|   2|[0.00164430072688...|
|        1|     3|   0|[0.01604230783414...|
|        1|     4|   0|[-0.0233688405714...|
|        2|     5|   0|[0.00142346554299...|
|        2

In [ ]:
assembler = VectorAssembler(
    inputCols=["reviewText_encoded", "rating", "vote"],
    outputCol="features"
)

pipeline = Pipeline(stages=[assembler])
model_pipe = pipeline.fit(reviews)
data = model_pipe.transform(reviews)

data = data.select("sentiment", "features")
data = data.withColumnRenamed("sentiment", "label")

(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=42)
testData.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(22,[20,21],[1.0,...|
|    0|[-0.0294870837325...|
|    0|[-0.0273159162715...|
|    0|[-0.0247566771189...|
|    0|[-0.0225437566945...|
|    0|[-0.0217187550098...|
|    0|[-0.0206075594954...|
|    0|[-0.0176324048508...|
|    0|[-0.0171053386135...|
|    0|[-0.0170503992333...|
|    0|[-0.0165446036317...|
|    0|[-0.0155462706057...|
|    0|[-0.0151897570739...|
|    0|[-0.0150075938723...|
|    0|[-0.0147838844452...|
|    0|[-0.0139539082135...|
|    0|[-0.0138143654554...|
|    0|[-0.0106316566719...|
|    0|[-0.0099415687449...|
|    0|[-0.0092599837499...|
+-----+--------------------+
only showing top 20 rows



#### RandomForest Classifier

In [ ]:
rf = RandomForestClassifier(
    numTrees=10,
    maxDepth=3,
    seed=42,
)
rf.getMinWeightFractionPerNode()
model = rf.fit(trainingData)

result = model.transform(testData)
result

label,features,rawPrediction,probability,prediction
0,"(22,[20,21],[1.0,...",[2.46871748771174...,[0.24687174877117...,2.0
0,[-0.0294870837325...,[5.27772584972332...,[0.52777258497233...,0.0
0,[-0.0273159162715...,[1.78845549923702...,[0.17884554992370...,2.0
0,[-0.0247566771189...,[1.71508350455295...,[0.17150835045529...,2.0
0,[-0.0225437566945...,[1.84088995616586...,[0.18408899561658...,2.0
0,[-0.0217187550098...,[1.52210947752094...,[0.15221094775209...,2.0
0,[-0.0206075594954...,[1.81711366443856...,[0.18171136644385...,2.0
0,[-0.0176324048508...,[5.89681491871158...,[0.58968149187115...,0.0
0,[-0.0171053386135...,[1.14558234015817...,[0.11455823401581...,2.0
0,[-0.0170503992333...,[4.78882953362112...,[0.47888295336211...,0.0


In [ ]:
predictionAndLabels = result.select("prediction", "label")
predictionAndLabels = predictionAndLabels \
    .withColumn("label", F.col("label").cast("double")) \
    .withColumn("prediction", F.col("prediction").cast("double")) \
    .rdd
metrics = MulticlassMetrics(predictionAndLabels)

print("RandomForestClassifier F1 score for class -1:\t{}" \
      .format(metrics.fMeasure(0.)))
print("RandomForestClassifier F1 score for class  0:\t{}" \
      .format(metrics.fMeasure(1.)))
print("RandomForestClassifier F1 score for class  1:\t{}" \
      .format(metrics.fMeasure(2.)))

RandomForestClassifier F1 score for class -1:	0.6666666666666667
RandomForestClassifier F1 score for class  0:	0.0
RandomForestClassifier F1 score for class  1:	0.9041095890410958


#### MultiLayerPerceptron Classifier

In [ ]:
layers = [22, 10, 10, 3]

mlpc = MultilayerPerceptronClassifier(
    maxIter=100,
    layers=layers,
    blockSize=128,
    seed=42
)
model = mlpc.fit(trainingData)

result = model.transform(testData)
result

label,features,rawPrediction,probability,prediction
0,"(22,[20,21],[1.0,...",[1.48188798667968...,[0.83569045654166...,0.0
0,[-0.0294870837325...,[1.47776685371858...,[0.83502846346514...,0.0
0,[-0.0273159162715...,[-0.6555775430423...,[0.21334753677978...,2.0
0,[-0.0247566771189...,[-0.5627280024258...,[0.24040244300000...,2.0
0,[-0.0225437566945...,[-0.6174611255184...,[0.22426169080419...,2.0
0,[-0.0217187550098...,[-1.0857399930204...,[0.11341922121988...,2.0
0,[-0.0206075594954...,[-1.2593306289127...,[0.08496280266678...,2.0
0,[-0.0176324048508...,[0.83441051180466...,[0.69707558201022...,0.0
0,[-0.0171053386135...,[-1.8395002495465...,[0.02889359203004...,2.0
0,[-0.0170503992333...,[2.02691005323134...,[0.90606016853204...,0.0


In [ ]:
predictionAndLabels = result.select("prediction", "label")
predictionAndLabels = predictionAndLabels \
    .withColumn("label", F.col("label").cast("double")) \
    .withColumn("prediction", F.col("prediction").cast("double")) \
    .rdd
metrics = MulticlassMetrics(predictionAndLabels)

print("Multilayer perceptron classifier F1 score for class -1:\t{}" \
      .format(metrics.fMeasure(0.)))
print("Multilayer perceptron classifier F1 score for class  0:\t{}" \
      .format(metrics.fMeasure(1.)))
print("Multilayer perceptron classifier F1 score for class  1:\t{}" \
      .format(metrics.fMeasure(2.)))

Multilayer perceptron classifier F1 score for class -1:	0.7428571428571429
Multilayer perceptron classifier F1 score for class  0:	0.0
Multilayer perceptron classifier F1 score for class  1:	0.9141274238227147


The two classifiers that were evaluated were the Random Forest and Multilayer Perceptron Classifier. When it comes to predicting positive comments, both classifiers demonstrated comparable performance with an F1 score of around 91%. This means that they were able to correctly identify and classify positive comments with high accuracy, indicating good results.

However, when it comes to predicting negative comments, the Multilayer Perceptron Classifier outperformed the Random Forest classifier with an F1 score of 71% compared to 67%. This indicates that the Multilayer Perceptron Classifier may be more accurate in predicting negative comments than the Random Forest classifier.

Considering the performance of the two classifiers, it seems that the Multilayer Perceptron Classifier would be a better choice if the focus is on predicting both positive and negative comments, given its higher accuracy in predicting negative comments and comparable accuracy in predicting positive comments. However, if predicting positive comments is the main focus, both classifiers demonstrate similar performance and either one could be chosen.

### Task 3. Matrix factorization based collaborative filtering

#### Make URM

In [ ]:
clear_all()
read_files()

In [ ]:
reviews = reviews \
    .drop("sentiment", "reviewTime", "reviewerName", "reviewText", "unixReviewTime", "vote", "image")
reviews.show()

+----------+------+--------------+
|product_id|rating|    reviewerID|
+----------+------+--------------+
|0439381673|     5| A1F1QP9FYQEVU|
|0439381673|     1| AC4SHUM57T99K|
|0439381673|     1|A1OT385HNQ7KVP|
|0439381673|     1|A3OU11ZFNAGKL8|
|0439381673|     1|A1WEOXIAQEBWZR|
|0439381673|     5|A1E6AXXMDYRFS3|
|0439381673|     3|A1404782FUNKF7|
|0439381673|     1| AR9984T786QGT|
|0439381673|     3| A4480YFBII5BF|
|0439381673|     5|A3G8LSDLKAMFMR|
|0439381673|     5| ARNF05LJD98X2|
|0439381673|     1|A2B79FG0GBD5ZW|
|0439381673|     4|A2OEZWDEDY19L0|
|0439381673|     1| A4L3IOTGCR9NW|
|0439381673|     1|A2EQI1KQ8BNVBN|
|0439381673|     5| A9OSOC53QERJ1|
|0439381673|     3|A1E2L47OGJVWOW|
|0439381673|     4|A23JCKJ9AHXRZZ|
|0439381673|     5|A1MN8PP39IIB7N|
|0439381673|     5|A1Q7VM39KORQDG|
+----------+------+--------------+
only showing top 20 rows



In [ ]:
string_indexer = StringIndexer(inputCol="product_id", outputCol="product_id_enc")
string_indexer = string_indexer.fit(reviews)
reviews = string_indexer.transform(reviews)
reviews = reviews.drop("product_id")
reviews.limit(5).show()

,rating,reviewerID,product_id_enc
0,5,A1F1QP9FYQEVU,3.0
1,1,AC4SHUM57T99K,3.0
2,1,A1OT385HNQ7KVP,3.0
3,1,A3OU11ZFNAGKL8,3.0
4,1,A1WEOXIAQEBWZR,3.0


In [ ]:
string_indexer = StringIndexer(inputCol="reviewerID", outputCol="reviewerID_enc")
string_indexer = string_indexer.fit(reviews)
reviews = string_indexer.transform(reviews)
reviews = reviews.drop("reviewerID")
reviews.limit(5).show()

,rating,product_id_enc,reviewerID_enc
0,5,3.0,163.0
1,1,3.0,629.0
2,1,3.0,199.0
3,1,3.0,564.0
4,1,3.0,240.0


In [ ]:
(training, test) = reviews.randomSplit([0.6, 0.4], seed=42)

In [ ]:
als = ALS(maxIter=10, regParam=0.1, userCol="reviewerID_enc",
          itemCol="product_id_enc", ratingCol="rating", 
          coldStartStrategy="drop", seed=42, nonnegative=True)
model = als.fit(training)

In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
# userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
# productsRecs = model.recommendForAllItems(10)

Root-mean-square error = 2.7853135855354862


In [ ]:
model.recommend

In [ ]:
test.summary(), predictions.summary()

(+-------+------------------+-----------------+------------------+
 |summary|            rating|   product_id_enc|    reviewerID_enc|
 +-------+------------------+-----------------+------------------+
 |  count|               324|              324|               324|
 |   mean| 4.219135802469136|39.41358024691358|            321.75|
 | stddev|1.3347479882085995|57.70090974399418|229.82193100102575|
 |    min|                 1|              0.0|               0.0|
 |    25%|                 4|              1.0|              99.0|
 |    50%|                 5|              8.0|             300.0|
 |    75%|                 5|             57.0|             527.0|
 |    max|                 5|            227.0|             745.0|
 +-------+------------------+-----------------+------------------+,
 +-------+------------------+------------------+------------------+------------------+
 |summary|            rating|    product_id_enc|    reviewerID_enc|        prediction|
 +-------+-----------

In [ ]:
predictions.show()

+------+--------------+--------------+-----------+
|rating|product_id_enc|reviewerID_enc| prediction|
+------+--------------+--------------+-----------+
|     5|          31.0|          32.0|  1.2277416|
|     5|          85.0|          77.0|  1.4156864|
|     4|          28.0|          84.0|  3.0759797|
|     5|          27.0|          26.0|  4.9534273|
|     4|          12.0|          92.0|0.008777629|
|     5|           1.0|          31.0|        0.0|
|     5|           1.0|          76.0| 0.09482071|
|     5|           1.0|           9.0|  2.2412002|
|     5|          52.0|          61.0|  1.2343442|
|     5|          13.0|           0.0|  1.2824899|
|     5|           6.0|          63.0|   4.953569|
|     5|           6.0|          14.0|   4.953569|
|     5|           9.0|           1.0|  2.8537717|
|     5|           9.0|          90.0|   1.249845|
|     5|           4.0|          57.0|  3.2205157|
|     4|           8.0|          51.0|   4.868396|
|     4|          23.0|        

#### Test a model for a specific user

In [ ]:
user = predictions.limit(1).select("reviewerID_enc").collect()[0].reviewerID_enc
user

32.0

In [ ]:
user_prods = reviews \
    .select("product_id_enc") \
    .distinct() \
    .join(
        reviews \
        .filter(f"reviewerID_enc={user}") \
        .select("reviewerID_enc") \
        .distinct(),
        how="full"
    )
user_prods.summary()

summary,product_id_enc,reviewerID_enc
count,228,228
mean,113.5,32.0
stddev,65.96211033616191,0.0
min,0.0,32.0
25%,56.0,32.0
50%,113.0,32.0
75%,170.0,32.0
max,227.0,32.0


In [ ]:
user_prods_res = model.transform(user_prods)
user_prods_res.show()

+--------------+--------------+-----------+
|product_id_enc|reviewerID_enc| prediction|
+--------------+--------------+-----------+
|         148.0|          32.0|  0.8726632|
|          31.0|          32.0|  1.2277416|
|          85.0|          32.0|  0.7319121|
|         137.0|          32.0|  1.5195677|
|          65.0|          32.0|  0.8015663|
|          53.0|          32.0|  1.9390309|
|         133.0|          32.0| 0.73805404|
|          78.0|          32.0|  1.8188006|
|         108.0|          32.0| 0.23217991|
|         155.0|          32.0| 0.73850644|
|          34.0|          32.0|  0.4333231|
|         211.0|          32.0|  1.3223755|
|         101.0|          32.0|  3.1739218|
|         126.0|          32.0|   2.366138|
|          28.0|          32.0|0.036129735|
|         210.0|          32.0|  3.1729982|
|          76.0|          32.0|  0.3602208|
|          26.0|          32.0|  2.4231746|
|          27.0|          32.0|  1.8188007|
|          44.0|          32.0| 

In [ ]:
top_5_products = user_prods_res \
    .sort(
        F.desc("prediction")
    ) \
    .limit(5) \
    .select("product_id_enc")
top_5_products.show()

+--------------+
|product_id_enc|
+--------------+
|         225.0|
|          74.0|
|         203.0|
|          48.0|
|          75.0|
+--------------+



#### Suggest theese products to 10 users

In [ ]:
ten_users = reviews.select("reviewerID_enc").distinct().limit(10)
ten_users.show()

+--------------+
|reviewerID_enc|
+--------------+
|         692.0|
|         299.0|
|         496.0|
|         305.0|
|         558.0|
|         596.0|
|         576.0|
|         170.0|
|         147.0|
|         720.0|
+--------------+



In [ ]:
top_5_products.join(ten_users, how="full").show()

+--------------+--------------+
|product_id_enc|reviewerID_enc|
+--------------+--------------+
|         225.0|         692.0|
|          74.0|         692.0|
|         203.0|         692.0|
|          48.0|         692.0|
|          75.0|         692.0|
|         225.0|         299.0|
|          74.0|         299.0|
|         203.0|         299.0|
|          48.0|         299.0|
|          75.0|         299.0|
|         225.0|         496.0|
|          74.0|         496.0|
|         203.0|         496.0|
|          48.0|         496.0|
|          75.0|         496.0|
|         225.0|         305.0|
|          74.0|         305.0|
|         203.0|         305.0|
|          48.0|         305.0|
|          75.0|         305.0|
+--------------+--------------+
only showing top 20 rows



In [ ]:
recs = model.transform(top_5_products.join(ten_users, how="full"))
recs.show()

+--------------+--------------+-----------+
|product_id_enc|reviewerID_enc| prediction|
+--------------+--------------+-----------+
|         225.0|         496.0|   4.547726|
|         225.0|         596.0|  2.4386578|
|         225.0|         305.0| 0.66240877|
|         225.0|         692.0|6.452682E-4|
|         225.0|         576.0|0.001075447|
|         225.0|         170.0|   4.547726|
|         225.0|         147.0|0.013336167|
|          48.0|         496.0|   4.286993|
|          48.0|         596.0|    2.33077|
|          48.0|         305.0|  1.8108616|
|          48.0|         692.0|  1.1577286|
|          48.0|         576.0|  1.9295474|
|          48.0|         170.0|   4.286993|
|          48.0|         147.0|   1.960142|
|          75.0|         496.0|  3.7974074|
|          75.0|         596.0|  1.9526116|
|          75.0|         305.0|0.123872064|
|          75.0|         692.0|  0.9462824|
|          75.0|         576.0|  1.5771374|
|          75.0|         170.0| 

In [ ]:
recs.join(test, on=["reviewerID_enc", "product_id_enc"], how="inner").show()

+--------------+--------------+----------+------+
|reviewerID_enc|product_id_enc|prediction|rating|
+--------------+--------------+----------+------+
+--------------+--------------+----------+------+



#### Top 5 results and metrics for it

In [ ]:
URM = predictions.sort(F.desc("prediction")).limit(5)
URM.show()

+------+--------------+--------------+----------+
|rating|product_id_enc|reviewerID_enc|prediction|
+------+--------------+--------------+----------+
|     5|           6.0|          14.0|  4.953569|
|     5|           6.0|          63.0|  4.953569|
|     5|          27.0|          26.0| 4.9534273|
|     5|           2.0|          28.0| 4.8991847|
|     5|           2.0|          91.0| 4.8991847|
+------+--------------+--------------+----------+



In [ ]:
# top_5_products = test.groupBy("product_id_enc")\
#     .agg(
#         F.avg("rating").alias("rating_avg")
#     ) \
#     .sort(F.desc("rating_avg")) \
#     .select("product_id_enc") \
#     .limit(5)
# top_5_products.show()

In [ ]:
# first_10_users = test.select("reviewerID_enc").distinct().sample(1.0, seed=42).limit(10)
# first_10_users.show()

In [ ]:
# URM = first_10_users.join(top_5_products, how="fullouter").withColumn("rating", F.lit(True))
# preds = model.transform(URM)
# preds.show()

In [ ]:
# Test the model for a specific user

# predictions.filter(F.col("reviewerID_enc") == 26.)

In [ ]:
temp = [
    ([5.0, 5.0, 5.0, 5.0, 5.0], [4.953569, 4.953569, 4.9534273, 4.8991847, 4.8991847])
]
dataset = spark.createDataFrame(temp, ["rating", "prediction"])
dataset.show()

+--------------------+--------------------+
|              rating|          prediction|
+--------------------+--------------------+
|[5.0, 5.0, 5.0, 5...|[4.953569, 4.9535...|
+--------------------+--------------------+



In [ ]:
from pyspark.ml.evaluation import RankingEvaluator

# URM = URM \
#     .withColumn("rating",
#         F.split(F.regexp_replace("rating", "[^0-9.,]", ""), ",").cast("array<double>")
#     ) \
#     .withColumn("prediction",
#         F.split(F.regexp_replace("prediction", "[^0-9.,]", ""), ",").cast("array<double>")
#     )

rval = RankingEvaluator(
    predictionCol="prediction", 
    labelCol="rating",
    metricName="precisionAtK"
)
rval.evaluate(dataset)

0.0

In [ ]:
rval = RankingEvaluator(
    predictionCol="prediction", 
    labelCol="rating", 
    metricName="recallAtK"
)
rval.evaluate(dataset)

0.0

In [ ]:
# scoreAndLabels = [([1.0, 6.0, 2.0, 7.0, 8.0, 3.0, 9.0, 10.0, 4.0, 5.0],
#     [1.0, 2.0, 3.0, 4.0, 5.0]),
#     ([4.0, 1.0, 5.0, 6.0, 2.0, 7.0, 3.0, 8.0, 9.0, 10.0], [1.0, 2.0, 3.0]),
#     ([1.0, 2.0, 3.0, 4.0, 5.0], [])]
# dataset = spark.createDataFrame(scoreAndLabels, ["prediction", "label"])
# dataset.show(truncate=False)

### Task 4. Sentiment-enhanced collaborative filtering

In [ ]:
clear_all()
read_files()

Unpersisted 1249 rdd
Unpersisted 1244 rdd


In [ ]:
reviews = reviews \
    .drop("reviewTime", "reviewerName", "reviewText", "unixReviewTime", "vote", "image")

string_indexer = StringIndexer(inputCol="reviewerID", outputCol="reviewerID_enc")
string_indexer = string_indexer.fit(reviews)
reviews = string_indexer.transform(reviews)
reviews = reviews.drop("reviewerID")

string_indexer = StringIndexer(inputCol="product_id", outputCol="product_id_enc")
string_indexer = string_indexer.fit(reviews)
reviews = string_indexer.transform(reviews)
reviews = reviews.drop("product_id")

reviews.show()

+---------+------+--------------+--------------+
|sentiment|rating|reviewerID_enc|product_id_enc|
+---------+------+--------------+--------------+
|        1|     5|         163.0|           3.0|
|        1|     1|         629.0|           3.0|
|        0|     1|         199.0|           3.0|
|       -1|     1|         564.0|           3.0|
|       -1|     1|         240.0|           3.0|
|        1|     5|         156.0|           3.0|
|        0|     3|         114.0|           3.0|
|       -1|     1|         701.0|           3.0|
|       -1|     3|         598.0|           3.0|
|        1|     5|         520.0|           3.0|
|        1|     5|         703.0|           3.0|
|       -1|     1|         315.0|           3.0|
|        1|     4|         380.0|           3.0|
|       -1|     1|         602.0|           3.0|
|       -1|     1|         329.0|           3.0|
|        1|     5|         624.0|           3.0|
|        0|     3|         155.0|           3.0|
|        0|     4|  

In [ ]:
URM = reviews \
    .drop("sentiment") \
    .select("reviewerID_enc") \
    .distinct() \
    .join(reviews.select("product_id_enc").distinct(), how="fullouter")
URM.show()

+--------------+--------------+
|reviewerID_enc|product_id_enc|
+--------------+--------------+
|         692.0|         147.0|
|         692.0|         170.0|
|         692.0|         184.0|
|         692.0|         160.0|
|         692.0|         169.0|
|         692.0|           8.0|
|         692.0|          67.0|
|         692.0|          70.0|
|         692.0|         168.0|
|         692.0|           0.0|
|         692.0|          69.0|
|         692.0|         206.0|
|         692.0|           7.0|
|         692.0|         142.0|
|         692.0|         191.0|
|         692.0|         112.0|
|         692.0|         154.0|
|         692.0|         124.0|
|         692.0|         128.0|
|         692.0|         201.0|
+--------------+--------------+
only showing top 20 rows



In [ ]:
results = model.transform(URM)
results.show()

+--------------+--------------+------------+
|reviewerID_enc|product_id_enc|  prediction|
+--------------+--------------+------------+
|         496.0|         148.0|2.5208446E-4|
|         243.0|         148.0|         0.0|
|         392.0|         148.0|    2.160591|
|         623.0|         148.0|         0.0|
|         737.0|         148.0|2.0166756E-4|
|          31.0|         148.0|         0.0|
|         516.0|         148.0|  0.94058245|
|          85.0|         148.0|    3.865726|
|         137.0|         148.0|   0.8726632|
|         451.0|         148.0|   2.0816576|
|         580.0|         148.0|2.5208446E-4|
|          65.0|         148.0|   1.4521642|
|         458.0|         148.0|   3.0775115|
|          53.0|         148.0|         0.0|
|         255.0|         148.0|2.5208446E-4|
|         481.0|         148.0|2.5208446E-4|
|          78.0|         148.0|   2.9683588|
|         322.0|         148.0|2.5208446E-4|
|         321.0|         148.0|  0.43520612|
|         

In [ ]:
inner_join = results \
    .withColumnRenamed("reviewerID_enc", "user_1") \
    .withColumnRenamed("prediction", "pred_1") \
    .join(
        results \
        .withColumnRenamed("reviewerID_enc", "user_2") \
        .withColumnRenamed("prediction", "pred_2"),
        on="product_id_enc", how="inner"
    )
inner_join.show()

+--------------+------+---------+------+----------+
|product_id_enc|user_1|   pred_1|user_2|    pred_2|
+--------------+------+---------+------+----------+
|         147.0| 496.0|4.0556564| 496.0| 4.0556564|
|         147.0| 496.0|4.0556564| 243.0|0.31210807|
|         147.0| 496.0|4.0556564| 392.0|    2.4405|
|         147.0| 496.0|4.0556564| 623.0| 3.4032707|
|         147.0| 496.0|4.0556564| 737.0|  3.244525|
|         147.0| 496.0|4.0556564|  31.0| 1.1490725|
|         147.0| 496.0|4.0556564| 516.0| 1.1409844|
|         147.0| 496.0|4.0556564|  85.0|       0.0|
|         147.0| 496.0|4.0556564| 137.0| 2.9116564|
|         147.0| 496.0|4.0556564| 451.0| 3.2747326|
|         147.0| 496.0|4.0556564| 580.0| 4.0556564|
|         147.0| 496.0|4.0556564|  65.0| 3.6291518|
|         147.0| 496.0|4.0556564| 458.0| 1.3945665|
|         147.0| 496.0|4.0556564|  53.0|  2.280278|
|         147.0| 496.0|4.0556564| 255.0| 4.0556564|
|         147.0| 496.0|4.0556564| 481.0| 4.0556564|
|         14

In [ ]:
    # .filter(F.col("user_1") != F.col("user_2")) \
inner_join = inner_join \
    .withColumn("diff", F.col("pred_1") - F.col("pred_2")) \
    .withColumn("2", F.lit(2)) \
    .withColumn("pwd", F.pow(F.col("diff"), F.col("2"))) \
    .drop("diff") \
    .drop("2")
inner_join.show()

+--------------+------+---------+------+----------+-------------------+
|product_id_enc|user_1|   pred_1|user_2|    pred_2|                pwd|
+--------------+------+---------+------+----------+-------------------+
|         147.0| 496.0|4.0556564| 496.0| 4.0556564|                0.0|
|         147.0| 496.0|4.0556564| 243.0|0.31210807|    14.014154572601|
|         147.0| 496.0|4.0556564| 392.0|    2.4405|   2.60873023562732|
|         147.0| 496.0|4.0556564| 623.0| 3.4032707|0.42560711679107044|
|         147.0| 496.0|4.0556564| 737.0|  3.244525| 0.6579342735576574|
|         147.0| 496.0|4.0556564|  31.0| 1.1490725|   8.44822930510054|
|         147.0| 496.0|4.0556564| 516.0| 1.1409844|  8.495312272339106|
|         147.0| 496.0|4.0556564|  85.0|       0.0| 16.448349103389774|
|         147.0| 496.0|4.0556564| 137.0| 2.9116564| 1.3087361221923857|
|         147.0| 496.0|4.0556564| 451.0| 3.2747326| 0.6098420491653087|
|         147.0| 496.0|4.0556564| 580.0| 4.0556564|             

In [ ]:
sim_scores = inner_join \
    .withColumn("half", F.lit(0.5)) \
    .groupBy(["user_1", "user_2", "half"]) \
    .agg(
        F.pow(F.sum("pwd") / F.count("pwd"), F.col("half")).alias("euc_dist")
    ) \
    .drop("half")
sim_scores.show()
    # .agg(
    #     F.collect_set("pred_1").alias("preds_1"),
    #     F.collect_set("pred_2").alias("preds_2")
    # ) \

+------+------+------------------+
|user_1|user_2|          euc_dist|
+------+------+------------------+
| 496.0| 442.0|3.2600056274310827|
| 496.0| 622.0| 2.725803173944918|
| 496.0|  83.0| 2.208070618282808|
| 496.0| 525.0| 3.252081294382491|
| 243.0| 645.0|2.9763991202800835|
| 243.0| 342.0|2.7887012524486927|
| 392.0| 496.0| 2.727257560805821|
| 392.0| 327.0| 2.377495594708992|
| 623.0| 409.0| 2.029509289602379|
| 623.0| 538.0|1.2975473230722043|
| 623.0| 441.0|2.2673345675682954|
| 623.0| 343.0|1.1676399467470369|
| 737.0| 636.0|1.2453133394331155|
| 737.0| 454.0|1.4016662634537311|
|  31.0| 358.0|1.2125360152220783|
|  31.0| 239.0|2.4883588192092794|
|  31.0| 345.0|2.4883588192092794|
|  31.0| 708.0|0.7660319224004953|
| 516.0| 377.0|1.3165352894016162|
| 516.0| 154.0|1.6392939994350806|
+------+------+------------------+
only showing top 20 rows



In [ ]:
# clear_cache()
score_cf = sim_scores \
    .join(inner_join.drop("pwd"), on=["user_1", "user_2"], how="inner")
score_cf.show()

+------+------+------------------+--------------+----------+------------+
|user_1|user_2|          euc_dist|product_id_enc|    pred_1|      pred_2|
+------+------+------------------+--------------+----------+------------+
|   0.0| 518.0|1.6488318371589168|         147.0| 1.5049617|   3.5737722|
|   0.0| 518.0|1.6488318371589168|         160.0| 0.4075563|   0.2771405|
|   0.0| 518.0|1.6488318371589168|           8.0| 2.1034427|   2.3090456|
|   0.0| 518.0|1.6488318371589168|          70.0| 3.6808867|   3.2267797|
|   0.0| 518.0|1.6488318371589168|           0.0|  2.412917|  0.92066413|
|   0.0| 518.0|1.6488318371589168|          69.0| 1.2826798|   3.2039485|
|   0.0| 518.0|1.6488318371589168|         206.0| 1.1574016|   1.5715874|
|   0.0| 518.0|1.6488318371589168|           7.0| 1.2657714|   2.6681986|
|   0.0| 518.0|1.6488318371589168|         191.0| 2.3568885|   2.1583977|
|   0.0| 518.0|1.6488318371589168|         154.0| 2.1922567|  0.59029555|
|   0.0| 518.0|1.6488318371589168|    

In [ ]:
score_cf = score_cf \
    .groupBy(["user_1", "product_id_enc"]) \
    .agg(
        F.sum(F.col("euc_dist") * F.col("pred_2")).alias("score_cf_t_p")
    )
score_cf.show()

+------+--------------+------------------+
|user_1|product_id_enc|      score_cf_t_p|
+------+--------------+------------------+
|   0.0|         114.0|1656.2037022520478|
|   1.0|         101.0|1515.6041076386848|
|   1.0|         199.0| 712.6100379843306|
|   2.0|         108.0|471.20503144026264|
|   2.0|         101.0| 1592.185322180169|
|   2.0|          92.0| 685.1188231047994|
|   3.0|          64.0| 1502.559672846226|
|   5.0|          54.0|509.22890270595036|
|   7.0|          71.0|  1617.12061601953|
|   7.0|          32.0| 590.7025958851144|
|   8.0|          77.0|1368.7204488930172|
|   8.0|         134.0| 849.5779094635719|
|   8.0|           3.0| 968.0443132554178|
|  12.0|         167.0|2179.1407296296356|
|  12.0|          16.0|1621.7165827060217|
|  14.0|          40.0|2171.0484070268085|
|  20.0|          62.0|1739.8997216873136|
|  23.0|          16.0|1658.5304137723354|
|  25.0|          79.0|1590.5192042424542|
|  28.0|         139.0|2827.9442255403223|
+------+---

In [ ]:
score_sa_p = reviews \
    .groupBy("product_id_enc") \
    .agg(
       (F.sum("sentiment") / F.count("sentiment")).alias("score_sa_p")
    )
score_sa_p.show()

+--------------+------------------+
|product_id_enc|        score_sa_p|
+--------------+------------------+
|         147.0|               1.0|
|         170.0|               1.0|
|         184.0|               1.0|
|         160.0|              -1.0|
|         169.0|               1.0|
|           8.0|0.5555555555555556|
|          67.0|               0.0|
|          70.0|               1.0|
|         168.0|               1.0|
|           0.0|0.7458563535911602|
|          69.0|               1.0|
|         206.0|               1.0|
|           7.0|0.7272727272727273|
|         142.0|               1.0|
|         191.0|               1.0|
|         112.0|               1.0|
|         154.0|               1.0|
|         124.0|               1.0|
|         128.0|               1.0|
|         201.0|               1.0|
+--------------+------------------+
only showing top 20 rows



In [ ]:
score_fin = score_cf \
    .join(score_sa_p, on="product_id_enc", how="inner")
score_fin.show()

+--------------+------+------------------+----------+
|product_id_enc|user_1|      score_cf_t_p|score_sa_p|
+--------------+------+------------------+----------+
|         147.0| 257.0|1892.3283211533308|       1.0|
|         147.0| 476.0|1829.2608330212165|       1.0|
|         147.0|  64.0|  2042.96474308168|       1.0|
|         147.0| 138.0|2147.9519510726077|       1.0|
|         147.0| 581.0|2451.4802017237944|       1.0|
|         147.0| 673.0| 2421.265852740559|       1.0|
|         147.0| 493.0|2416.6649725019497|       1.0|
|         147.0| 638.0| 2307.050832363693|       1.0|
|         147.0| 727.0|2307.0508323636936|       1.0|
|         147.0| 586.0|2322.7013928320875|       1.0|
|         147.0| 485.0|2653.6867110272806|       1.0|
|         147.0| 570.0|1990.5378086750559|       1.0|
|         147.0| 682.0|1964.7286449609503|       1.0|
|         147.0| 436.0|2307.0508323636927|       1.0|
|         147.0| 390.0|1719.9968207266797|       1.0|
|         147.0| 494.0|  241

In [ ]:
scores = score_fin \
    .withColumn("result", F.col("score_cf_t_p") * 0.8 + F.col("score_sa_p") * 0.2)
scores.show()

+--------------+------+------------------+----------+------------------+
|product_id_enc|user_1|      score_cf_t_p|score_sa_p|            result|
+--------------+------+------------------+----------+------------------+
|         147.0| 257.0|1892.3283211533308|       1.0|1514.0626569226647|
|         147.0| 476.0|1829.2608330212165|       1.0|1463.6086664169734|
|         147.0|  64.0|  2042.96474308168|       1.0| 1634.571794465344|
|         147.0| 138.0|2147.9519510726077|       1.0|1718.5615608580863|
|         147.0| 581.0|2451.4802017237944|       1.0|1961.3841613790357|
|         147.0| 673.0| 2421.265852740559|       1.0|1937.2126821924471|
|         147.0| 493.0|2416.6649725019497|       1.0|  1933.53197800156|
|         147.0| 638.0| 2307.050832363693|       1.0|1845.8406658909546|
|         147.0| 727.0|2307.0508323636936|       1.0| 1845.840665890955|
|         147.0| 586.0|2322.7013928320875|       1.0|1858.3611142656703|
|         147.0| 485.0|2653.6867110272806|       1.